In [67]:
import cv2
import numpy as np
import imgaug as ia 
import glob
import os
from keras.preprocessing import image


In [103]:
def dice_coef(img, img2):
        if img.shape != img2.shape:
            raise ValueError("Shape mismatch: img and img2 must have to be of the same shape.")
        
        else:
            lenIntersection=0
            for i in range(img.shape[0]):
                for j in range(img.shape[1]):
                    if (np.array_equal(img[i][j],img2[i][j])):
                        lenIntersection+=1
             
            lenimg=img.shape[0]*img.shape[1]
            lenimg2=img2.shape[0]*img2.shape[1]  
            value = (2. * lenIntersection  / (lenimg + lenimg2))
        return value

def iou_coef(inputs, target, smooth=1):
    intersection = (target * inputs).sum()
    union = target.sum() + inputs.sum() - intersection
    iou = np.sum(intersection) / np.sum(union)
    return iou

def iou(prediction, target):
    intersection = np.logical_and(target, prediction)
    union = np.logical_or(target, prediction)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

def dice(pred, true, k = 1):
    intersection = np.sum(pred[true==k]) * 2.0
    dice = intersection / (np.sum(pred) + np.sum(true))
    return dice

In [69]:
def pixel_accuracy(eval_segm, gt_segm):
    '''
    sum_i(n_ii) / sum_i(t_i)
    '''

    check_size(eval_segm, gt_segm)

    cl, n_cl = extract_classes(gt_segm)
    eval_mask, gt_mask = extract_both_masks(eval_segm, gt_segm, cl, n_cl)

    sum_n_ii = 0
    sum_t_i  = 0

    for i, c in enumerate(cl):
        curr_eval_mask = eval_mask[i, :, :]
        curr_gt_mask = gt_mask[i, :, :]

        sum_n_ii += np.sum(np.logical_and(curr_eval_mask, curr_gt_mask))
        sum_t_i  += np.sum(curr_gt_mask)
 
    if (sum_t_i == 0):
        pixel_accuracy_ = 0
    else:
        pixel_accuracy_ = sum_n_ii / sum_t_i

    return pixel_accuracy_

def mean_accuracy(eval_segm, gt_segm):
    '''
    (1/n_cl) sum_i(n_ii/t_i)
    '''

    check_size(eval_segm, gt_segm)

    cl, n_cl = extract_classes(gt_segm)
    eval_mask, gt_mask = extract_both_masks(eval_segm, gt_segm, cl, n_cl)

    accuracy = list([0]) * n_cl

    for i, c in enumerate(cl):
        curr_eval_mask = eval_mask[i, :, :]
        curr_gt_mask = gt_mask[i, :, :]

        n_ii = np.sum(np.logical_and(curr_eval_mask, curr_gt_mask))
        t_i  = np.sum(curr_gt_mask)
 
        if (t_i != 0):
            accuracy[i] = n_ii / t_i

    mean_accuracy_ = np.mean(accuracy)
    return mean_accuracy_

def mean_IU(eval_segm, gt_segm):
    '''
    (1/n_cl) * sum_i(n_ii / (t_i + sum_j(n_ji) - n_ii))
    '''

    check_size(eval_segm, gt_segm)

    cl, n_cl   = union_classes(eval_segm, gt_segm)
    _, n_cl_gt = extract_classes(gt_segm)
    eval_mask, gt_mask = extract_both_masks(eval_segm, gt_segm, cl, n_cl)

    IU = list([0]) * n_cl

    for i, c in enumerate(cl):
        curr_eval_mask = eval_mask[i, :, :]
        curr_gt_mask = gt_mask[i, :, :]
 
        if (np.sum(curr_eval_mask) == 0) or (np.sum(curr_gt_mask) == 0):
            continue

        n_ii = np.sum(np.logical_and(curr_eval_mask, curr_gt_mask))
        t_i  = np.sum(curr_gt_mask)
        n_ij = np.sum(curr_eval_mask)

        IU[i] = n_ii / (t_i + n_ij - n_ii)
 
    mean_IU_ = np.sum(IU) / n_cl_gt
    return mean_IU_

def frequency_weighted_IU(eval_segm, gt_segm):
    '''
    sum_k(t_k)^(-1) * sum_i((t_i*n_ii)/(t_i + sum_j(n_ji) - n_ii))
    '''

    check_size(eval_segm, gt_segm)

    cl, n_cl = union_classes(eval_segm, gt_segm)
    eval_mask, gt_mask = extract_both_masks(eval_segm, gt_segm, cl, n_cl)

    frequency_weighted_IU_ = list([0]) * n_cl

    for i, c in enumerate(cl):
        curr_eval_mask = eval_mask[i, :, :]
        curr_gt_mask = gt_mask[i, :, :]
 
        if (np.sum(curr_eval_mask) == 0) or (np.sum(curr_gt_mask) == 0):
            continue

        n_ii = np.sum(np.logical_and(curr_eval_mask, curr_gt_mask))
        t_i  = np.sum(curr_gt_mask)
        n_ij = np.sum(curr_eval_mask)

        frequency_weighted_IU_[i] = (t_i * n_ii) / (t_i + n_ij - n_ii)
 
    sum_k_t_k = get_pixel_area(eval_segm)
    
    frequency_weighted_IU_ = np.sum(frequency_weighted_IU_) / sum_k_t_k
    return frequency_weighted_IU_

'''
Auxiliary functions used during evaluation.
'''
def get_pixel_area(segm):
    return segm.shape[0] * segm.shape[1]

def extract_both_masks(eval_segm, gt_segm, cl, n_cl):
    eval_mask = extract_masks(eval_segm, cl, n_cl)
    gt_mask   = extract_masks(gt_segm, cl, n_cl)

    return eval_mask, gt_mask

def extract_classes(segm):
    cl = np.unique(segm)
    n_cl = len(cl)

    return cl, n_cl

def union_classes(eval_segm, gt_segm):
    eval_cl, _ = extract_classes(eval_segm)
    gt_cl, _   = extract_classes(gt_segm)

    cl = np.union1d(eval_cl, gt_cl)
    n_cl = len(cl)

    return cl, n_cl

def extract_masks(segm, cl, n_cl):
    h, w  = segm_size(segm)
    masks = np.zeros((n_cl, h, w))

    for i, c in enumerate(cl):
        masks[i, :, :] = segm == c

    return masks

def segm_size(segm):
    try:
        height = segm.shape[0]
        width  = segm.shape[1]
    except IndexError:
        raise

    return height, width

def check_size(eval_segm, gt_segm):
    h_e, w_e = segm_size(eval_segm)
    h_g, w_g = segm_size(gt_segm)

    if (h_e != h_g) or (w_e != w_g):
        raise EvalSegErr("DiffDim: Different dimensions of matrices!")

In [117]:
def UiP(pred, true):
    #pred = cv2.imread(pred, cv2.IMREAD_COLOR)
    pred = cv2.cvtColor(pred, cv2.COLOR_BGR2GRAY)
    (thresh, pred) = cv2.threshold(pred, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    #true = cv2.imread(true, cv2.IMREAD_COLOR)
    true = cv2.cvtColor(true, cv2.COLOR_BGR2GRAY)
    (thresh, true) = cv2.threshold(true, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    union = np.logical_or(pred, true)
    union = union.astype(np.uint8) * 255
    dif = cv2.subtract(union, true)

    return (1-cv2.countNonZero(dif) / dif.size)
        
def metrics(pred, true):
    uip_c = UiP(pred, true)
    IoU_c = iou(pred, true)
    dice_c = dice(pred, true, k = 255)
    dice_c2 = dice_coef(pred, true)
    
    return uip_c, IoU_c, dice_c, dice_c2

In [99]:
diretorio_rgb = "../../daninhas/dataset/test/rgb/"
diretorio_pred = "../../daninhas/dataset/test/prediction/"
diretorio_true = "../../daninhas/dataset/test/true/"

In [145]:
num_images = len(os.listdir(diretorio_pred))
uip_c, IoU_c, dice_c, dice_c2 = [0, 0, 0, 0]

c1 = 0 
c3 = 0
c4 = 0
c5 = 0

for name in os.listdir(diretorio_pred):    
    img_true = cv2.imread(diretorio_pred + name)
    img_pred = cv2.imread(diretorio_true + name)
    #[uip_c, iou_c, IoU_c, dice_c, dice_c2] += [uip_c, iou_c, IoU_c, dice_c, dice_c2]
    uip_c, IoU_c, dice_c, dice_c2 = metrics(img_pred, img_true)
    
    c1 += uip_c 
    c3 += IoU_c
    c4 += dice_c
    c5 += dice_c2
    

print( c1 / num_images, c3 / num_images, '\n',
      c4 / num_images, c5 / num_images)

0.9960863162309695 0.43997556460953025 
 0.4665112655159001 0.9306511512169471


In [155]:

print(' UiP :', c1 / num_images, '\n','IoU :', c3 / num_images,'\n',
      'Dice :', c4 / num_images, '\n', 'Dice_coef :', c5 / num_images)
img_t = cv2.imread('../../daninhas/dataset/test/true/daninha_229.jpg')
img_p = cv2.imread('../../daninhas/dataset/test/prediction/daninha_229.jpg')

metrics(img_p, img_t)

 UiP : 0.9960863162309695 
 IoU : 0.43997556460953025 
 Dice : 0.4665112655159001 
 Dice_coef : 0.9306511512169471


(0.886932373046875, 0.2190191508764819, 0.33594975416601275, 0.729034423828125)